In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
# geospatial
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

## Nashville Crime and <b>Community Amenities</b> Analysis

This project will be an examination of how crime and accessibility of community resources vary
across communities in Nashville and if there are any patterns or correlations to be found
between them. I believe that it could shed light on how underprivileged communities with less
access to amenities such as housing, green areas, and other resources influence crime rates.
The data to be used in this analysis will include local data to Nashville and Davidson county,
much of which is obtained through the Nashville Open Data Portal online.

<h5><b>Data Question</b></h5>
How does crime in Nashville vary across neighborhoods with different community resources,
and what correlations or patterns are there?

### Exploration & Cleaning

#### Greenways Data

In [3]:
greenways = gpd.read_file('../data/Greenways.geojson')

In [4]:
greenways.shape

(166, 12)

In [5]:
greenways.head()

,GlobalID,Difficulty,SurfaceMaterial,Greenway,GreenwayTrailName,Mileage,Description,OwnedBy,GreenwayType,YearBuilt,OBJECTID,geometry
0,9ab32307-9255-42bb-98b3-bcc9ee9e4577,None,Paved,Mill Creek Greenway,Orchard Bend Park,0.083224,Turner,None,Greenway,NaN,28,"LINESTRING Z (-86.66974 36.02465 0.00000, -86...."
1,05caea61-9112-419e-bb95-70c0284ee52d,,Paved,Cumberland River Greenway,Brookemeade Park,0.364287,"Trail To Kelley's Point, Cumberland River Over...",Metro,Multi-use,2005.0,494,MULTILINESTRING Z ((-86.90419 36.13075 0.00000...
2,09adeb86-5a30-449f-b88f-271c284dad9a,,Paved,Cumberland River Greenway,Downtown,0.650000,Trail connecting the riverfront to Bicentenial...,Metro,Multi-use,2001.0,495,MULTILINESTRING Z ((-86.78201 36.17455 0.00000...
3,569f7bb2-4333-4e31-8c52-471c454aa5a9,,Paved,Cumberland River Greenway,Downtown,0.542944,Rolling Mill Hill overlook. Connects to Ascend...,Metro,Multi-use,2013.0,496,"LINESTRING Z (-86.76469 36.15531 0.00000, -86...."
4,a401702f-5ba2-453c-a4eb-4907e61fa33d,,Paved,Cumberland River Greenway,Metro Center Levee - Catfish Tunnel to Freelan...,1.495068,Main Trail,Metro,Multi-use,2004.0,498,MULTILINESTRING Z ((-86.80091 36.20110 0.00000...


In [6]:
greenways['Mileage'].mean()

0.5055526892168675

In [7]:
greenways['GreenwayType'].value_counts()

GreenwayType
Multi-use     147
Primitive       8
Greenway        4
Park Trail      4
Multi use       1
Name: count, dtype: int64

In [8]:
greenways[greenways['SurfaceMaterial'] == 'Unpaved']

,GlobalID,Difficulty,SurfaceMaterial,Greenway,GreenwayTrailName,Mileage,Description,OwnedBy,GreenwayType,YearBuilt,OBJECTID,geometry
22,fab7c10b-4124-4455-aec9-2f1ca958da17,,Unpaved,Stones River Greenway,YMCA Connector,0.270000,Primitive Trail,Metro,Primitive,2005.0,527,MULTILINESTRING Z ((-86.64245 36.17312 0.00000...
73,c901e8ed-241f-446e-a050-e93f3cfb0204,,Unpaved,Seven Mile Creek Greenway,Ellington Segment,0.300000,Primitive Trail,Metro,Primitive,2008.0,587,"LINESTRING Z (-86.74239 36.05957 0.00000, -86...."
100,9d61407e-35a0-47e9-b554-f117aadfe712,Moderate,Unpaved,Cumberland River Greenway,Crooked Branch Park,0.363730,Primitive Trail,Metro,Primitive,2013.0,622,"LINESTRING Z (-86.64651 36.24079 0.00000, -86...."
101,e438f7e5-9958-4029-9dfc-f490168c8c2a,Moderate,Unpaved,Cumberland River Greenway,Crooked Branch Park,0.066660,Primitive Trail,Metro,Primitive,2013.0,623,"LINESTRING Z (-86.65017 36.23859 0.00000, -86...."
102,85142b4f-ae8a-4efa-b97a-bdb066b710f2,Moderate,Unpaved,Cumberland River Greenway,Crooked Branch Park,0.065186,Primitive Trail,Metro,Primitive,2013.0,624,"LINESTRING Z (-86.64885 36.24095 0.00000, -86...."
107,90a60d9a-62f4-4d3b-83e6-c79993bd48c2,,Unpaved,Shelby Bottoms Greenway,Cornelia Fort Connector - north,0.150000,Primitive Trail,Metro,Primitive,2013.0,630,"LINESTRING Z (-86.69688 36.19271 0.00000, -86...."
122,052a950b-03f9-4d74-a5fa-2b4ad3367b87,,Unpaved,Seven Mile Creek Greenway,Caldwell Park Trail,0.463627,Primitive Loop Trail,Metro,Primitive,2013.0,647,"LINESTRING Z (-86.72752 36.07779 0.00000, -86...."
158,09412bc8-15ef-41e6-abd9-1c8ac9620cff,None,Unpaved,Bells Bend Greenway,Dove Farm Loop Trail,1.940000,None,None,Primitive,NaN,3770,"LINESTRING Z (-86.92665 36.15633 0.00000, -86...."
159,c56e10e2-d9b0-4a4c-84f8-93cc968d0577,None,Unpaved,Bells Bend Greenway,River Loop Trail,2.450000,None,None,None,NaN,3777,MULTILINESTRING Z ((-86.92475 36.14349 0.00000...


##### Exploring

<b>Difficulty:</b> Difficulty of greenway traversal<br>
<b>Surface Material:</b> Greenways surface material<br>
<b>Greenway:</b> Greenway name<br>
<b>Greenway Segment:</b> Greenway segments name<br>
<b>Mileage:</b> Length of greenway segment in miles<br>
<b>Description:</b> Description of greenway segment<br>
<b>Owned By:</b> Owner of greenway segment<br>
<b>Greenway Type:</b> Type of greenway<br>
<b>Year Built:</b> The year that the greenway segment was constructed

#### Park Trails

In [9]:
trails = gpd.read_file('../data/Park_Trails_-3337017347785075052.geojson')

In [10]:
trails.head()

,GlobalID,TrailName,TrailType,Description,Mileage,Difficulty,SurfaceMaterial,YearBuilt,PARK_ID,OBJECTID,geometry
0,8be583ca-e2b5-4b84-898e-e1f6dac6d70f,Access Trail,Mountain Bike Trail,,0.108945,,Unpaved,NaN,PARK_172,1,"LINESTRING (-86.62429 36.10385, -86.62431 36.1..."
1,d7dda5a8-d7ce-4ff2-8b4f-8cc330c56a4f,East/West Connector,Mountain Bike Trail,,0.194277,,Unpaved,NaN,PARK_172,2,"LINESTRING (-86.62397 36.09706, -86.62403 36.0..."
2,94402a26-c5b4-436e-9dc9-c358ee680896,Cane Ridge Mountain Bike Trail,Mountain Bike Trail,,0.005279,,Unpaved,NaN,PARK_47,3,"LINESTRING (-86.61937 35.99142, -86.61928 35.9..."
3,38ef5ccb-3509-4f7c-aba8-b0c3f6d80c47,Cane Ridge Mountain Bike Trail,Mountain Bike Trail,,0.002665,,Unpaved,NaN,PARK_47,4,"LINESTRING (-86.61928 35.99140, -86.61923 35.9..."
4,3da17cbb-3094-4077-94a9-a21d09ad6928,Cane Ridge Mountain Bike Trail,Mountain Bike Trail,,0.023682,,Unpaved,NaN,PARK_47,5,"LINESTRING (-86.61923 35.99139, -86.61920 35.9..."


In [11]:
trails.shape

(238, 11)

In [12]:
trails['TrailType'].value_counts()

TrailType
Park Trail             80
Primitive              80
Mountain Bike Trail    35
Horse Trail            31
Multi-use              11
Name: count, dtype: int64

#### Combining Greenways and Park Trails

In [13]:
# Looking for common Columns
common_cols_a = []
for x in list(greenways.columns):
    if x in list(trails.columns):
        common_cols_a.append(x)
    else:
        print('DROP: ' + str(x))

DROP: Greenway
DROP: GreenwayTrailName
DROP: OwnedBy
DROP: GreenwayType


In [14]:
common_cols_a.sort()
print(common_cols_a)

['Description', 'Difficulty', 'GlobalID', 'Mileage', 'OBJECTID', 'SurfaceMaterial', 'YearBuilt', 'geometry']


In [15]:
# Checking Reverse
common_cols_b = []
for x in list(trails.columns):
    if x in list(greenways.columns):
        common_cols_b.append(x)
    else:
        print('DROP: ' + str(x))

DROP: TrailName
DROP: TrailType
DROP: PARK_ID


In [16]:
common_cols_b.sort()
print(common_cols_b)

['Description', 'Difficulty', 'GlobalID', 'Mileage', 'OBJECTID', 'SurfaceMaterial', 'YearBuilt', 'geometry']


In [17]:
# I do notice that I can simply rename 'GreenwayTrailName' in greenways and 'TrailName' in park trails to
# include them in my dataframe concatenation.
# Same thing with 'GreenwayType' and 'TrailType'
# Combining 'Greenway' and 'ParkID'?

In [18]:
# Peek Greenway and Trail types
greenways['GreenwayType'].unique()
# There are multiple strings for 'multi use' so need to clean that after combining them.

array(['Greenway', 'Multi-use', 'Park Trail', 'Primitive', None,
       'Multi use'], dtype=object)

In [19]:
trails['TrailType'].unique()

array(['Mountain Bike Trail', 'Park Trail', 'Primitive', 'Multi-use',
       'Horse Trail', None], dtype=object)

In [20]:
# Checking 'Greenway' and 'ParkID'

In [21]:
greenways['Greenway'].unique()

array(['Mill Creek Greenway', 'Cumberland River Greenway',
       'Shelby Bottoms Greenway', 'Forest Hills Greenway',
       'Gulch Greenway', 'Harpeth River Greenway',
       'Richland Creek Greenway', 'Stones River Greenway',
       'Peeler Park Greenway', 'Whites Creek Greenway',
       'Seven Mile Creek Greenway', 'Seven Mile Creek Greenway ',
       'Harpeth River Greenway ', '440 Greenway', 'Browns Creek Greenway',
       'Whites Creek Greenway-Fontanel', 'Bells Bend Greenway'],
      dtype=object)

In [22]:
trails['PARK_ID'].unique()

array(['PARK_172', 'PARK_47', 'PARK_50', 'PARK_3', 'PARK_43',
       'State Park', 'USACE', 'PARK_71', 'PARK_41', 'PARK_26', 'PARK_49',
       'PARK-00009', 'PARK_53', 'PARK_166', 'PARK_31', 'PARK_46',
       'PARK_1', 'PARK_67', 'State NA', 'PARK_80', 'PARK_70', 'PARK_161',
       'PARK_8', 'PARK_54', 'PARK_77', 'PARK_39', 'PARK_22', 'PARK_58',
       'PARK_36', 'PARK_200', 'PARK_21', 'PARK_174', 'PARK_57', 'PARK_19',
       'PARK_83', 'PARK_33', 'PARK_63', 'PARK_59', 'PARK_211', 'PARK_5',
       'PARK_62', None, 'PARK_4', 'PARK_88', 'PARK_223', 'PARK_212',
       'PARK_73', 'PARK_89', 'State Ag', 'PARK_217', 'Fontanel',
       'PARK_207', 'PARK_ID\tPARK', 'PARK_218', ' '], dtype=object)

In [23]:
# Do I integrate these into one column...

# Yes, they both function as specific identifiers for each installment. I would prefer to have these in one column, if they were in two, that would be another issue on its own.

In [24]:
# What's this... 
trails[trails['PARK_ID'] == 'PARK_ID\tPARK']

,GlobalID,TrailName,TrailType,Description,Mileage,Difficulty,SurfaceMaterial,YearBuilt,PARK_ID,OBJECTID,geometry
235,eb42a082-d62d-48ef-86f7-15a2a37f39ba,Mill Creek Greenway,Park Trail,Loop Trail,0.423188,,Paved,2014.0,PARK_ID\tPARK,2688,"LINESTRING (-86.68647 36.01685, -86.68693 36.0..."


In [25]:
trails[trails['TrailName'] == 'Mill Creek Greenway']

,GlobalID,TrailName,TrailType,Description,Mileage,Difficulty,SurfaceMaterial,YearBuilt,PARK_ID,OBJECTID,geometry
234,f40ac0b4-d34a-432f-b916-4abaf8876697,Mill Creek Greenway,Park Trail,Primitive Neighborhood Connector,0.077455,Easy,Unpaved,2013.0,PARK_207,2687,"LINESTRING (-86.68946 36.01725, -86.68948 36.0..."
235,eb42a082-d62d-48ef-86f7-15a2a37f39ba,Mill Creek Greenway,Park Trail,Loop Trail,0.423188,,Paved,2014.0,PARK_ID\tPARK,2688,"LINESTRING (-86.68647 36.01685, -86.68693 36.0..."
236,3b84d320-9b1c-4835-8655-febd914d6009,Mill Creek Greenway,Park Trail,None,0.248555,Easy,Paved,2019.0,PARK_218,2689,"LINESTRING (-86.67175 36.02712, -86.67143 36.0..."


In [26]:
# Weird but I don't think I can do much about it

In [27]:
# Renaming for concatenation
greenways = greenways.rename(columns={'GreenwayTrailName' : 'name',
                                      'GreenwayType' : 'type',
                                      'Greenway' : 'Installment'}).drop(columns='OwnedBy') # <<< Dropping one column I don't need
trails = trails.rename(columns={'TrailName' : 'name',
                                'TrailType' : 'type',
                                'PARK_ID' : 'Installment'})

In [28]:
# Check again
common_cols_a = []
for x in list(greenways.columns):
    if x in list(trails.columns):
        common_cols_a.append(x)
    else:
        print('DROP: ' + str(x))

common_cols_a

['GlobalID',
 'Difficulty',
 'SurfaceMaterial',
 'Installment',
 'name',
 'Mileage',
 'Description',
 'type',
 'YearBuilt',
 'OBJECTID',
 'geometry']

In [29]:
# Looks good! All other column names have the exact same spelling and capitalization so it's LAUNCH TIME!
greenways_trails = pd.concat([greenways, trails], ignore_index=True)

In [30]:
# Voila
greenways_trails.head()

,GlobalID,Difficulty,SurfaceMaterial,Installment,name,Mileage,Description,type,YearBuilt,OBJECTID,geometry
0,9ab32307-9255-42bb-98b3-bcc9ee9e4577,None,Paved,Mill Creek Greenway,Orchard Bend Park,0.083224,Turner,Greenway,NaN,28,"LINESTRING Z (-86.66974 36.02465 0.00000, -86...."
1,05caea61-9112-419e-bb95-70c0284ee52d,,Paved,Cumberland River Greenway,Brookemeade Park,0.364287,"Trail To Kelley's Point, Cumberland River Over...",Multi-use,2005.0,494,MULTILINESTRING Z ((-86.90419 36.13075 0.00000...
2,09adeb86-5a30-449f-b88f-271c284dad9a,,Paved,Cumberland River Greenway,Downtown,0.650000,Trail connecting the riverfront to Bicentenial...,Multi-use,2001.0,495,MULTILINESTRING Z ((-86.78201 36.17455 0.00000...
3,569f7bb2-4333-4e31-8c52-471c454aa5a9,,Paved,Cumberland River Greenway,Downtown,0.542944,Rolling Mill Hill overlook. Connects to Ascend...,Multi-use,2013.0,496,"LINESTRING Z (-86.76469 36.15531 0.00000, -86...."
4,a401702f-5ba2-453c-a4eb-4907e61fa33d,,Paved,Cumberland River Greenway,Metro Center Levee - Catfish Tunnel to Freelan...,1.495068,Main Trail,Multi-use,2004.0,498,MULTILINESTRING Z ((-86.80091 36.20110 0.00000...


#### Short Term Rentals

In [31]:
short_term_rentals = gpd.read_file('../data/Residential_Short_Term_Rental_Permits_view_9011625907413496580.geojson')

In [32]:
short_term_rentals.head(1)

,Permit__,Permit_Subtype_Description,Date_Entered,Date_Issued,Address,City,State,Subdivision_Lot,Permit_Type,Permit_SubType,Purpose,Council_Dist,Census_Tract,Expiration_Date,Permit_Owner_Name,Permit_Owner_Addr1,Permit_Owner_Addr2,Permit_Owner_Addr3,Permit_Owner_City,Permit_Owner_State,Permit_Owner_Zip,Permit_Status,Applicant,Applicant_Home_Phone,Applicant_Mobile_Phone,Applicant_Work_Phone,Applicant_Work_Phone_Ext,Contact,Contact_Home_Phone,Contact_Mobile_Phone,Contact_Work_Phone,Contact_Work_Phone_Ext,Lon,Lat,ObjectId,ZIP,Parcel,geometry
0,201511887,Short Term Rental - Non-Owner Occupied,"Wed, 01 Apr 2015 05:00:00 GMT","Thu, 09 Jul 2015 05:00:00 GMT",133 HAYNES PARK DR,NASHVILLE,TN,LOT 187 HAYNES PARK SUB SEC 2,CASR,CAZ10A002,By making this application for a Residential S...,1,37012801,"Tue, 08 Aug 2023 00:00:00 GMT",KCJB FAMILY TRUST,None,8210 MOORES LN,None,BRENTWOOD,TN,37027,EXPIRED,"Fox, Stephanie S.",None,None,None,None,"Fox, Stephanie S.",None,None,None,None,-86.834922,36.209292,1,37218,06904012500,POINT (-9666419.30900 4329457.98320)


In [33]:
# I don't need any contact information
short_term_rentals = short_term_rentals.drop(columns = ['Applicant_Home_Phone', 'Applicant_Mobile_Phone',
       'Applicant_Work_Phone', 'Applicant_Work_Phone_Ext',
       'Contact_Home_Phone', 'Contact_Mobile_Phone', 'Contact_Work_Phone',
       'Contact_Work_Phone_Ext'])

In [34]:
short_term_rentals['Permit_Status'].value_counts()

Permit_Status
ISSUED       7543
EXPIRED      6959
CANCELLED    1838
REVOKED        95
EXPIREEXT      37
PENDING        32
HOLD           19
REFUNDED        3
DENIED-D        2
DENIED          2
VOID            1
Name: count, dtype: int64

In [35]:
# Filtering for active Short Term Rental permits
active_STRs = short_term_rentals[short_term_rentals['Permit_Status'] == 'ISSUED']

In [36]:
# Checking
active_STRs['Permit_Status'].value_counts()

Permit_Status
ISSUED    7543
Name: count, dtype: int64

#### Nashville Next

QUESTIONS:
- What is the anchor park network and what are the specific plans for 'anchor parks?'
- Can you shed some light on what the tiers are? Are they a priority system?
- As far as features, what do Center, Transition or Infill, and Special Uses mean?

In [37]:
nash_next = gpd.read_file('../data/NashvilleNext_Concept_Map.geojson')

In [38]:
nash_next.head(10)

,OBJECTID,Name,Tier,Network,FeatureType,geometry
0,1,Anchor Parks,,Anchor Park,Anchor Park,"MULTIPOLYGON (((-86.87317 36.08708, -86.87312 ..."
1,2,East Nashville - Gallatin Rd/E Trinity Ln,First,,Center,"POLYGON ((-86.73471 36.20726, -86.73481 36.205..."
2,3,100 Oaks Area,First,,Center,"POLYGON ((-86.75155 36.11600, -86.75178 36.113..."
3,4,8th Ave S/Melrose/Berry Hill,First,,Center,"POLYGON ((-86.77257 36.12655, -86.77337 36.121..."
4,5,Antioch - Bell Rd/Murfreesboro Pk,Second,,Center,"POLYGON ((-86.63672 36.07483, -86.63642 36.074..."
5,6,Antioch - Crossings/Hickory Hollow,First,,Center,"MULTIPOLYGON (((-86.66036 36.05466, -86.66020 ..."
6,7,Bellevue - Bellevue Mall Area,Third,,Center,"POLYGON ((-86.94341 36.07690, -86.94350 36.076..."
7,8,Bellevue - Biltmore/McCrory/I40,Third,,Center,"POLYGON ((-87.01336 36.07744, -87.01336 36.077..."
8,9,Bellevue - Charlotte Pk/Old Hickory Blvd,Second,,Center,"POLYGON ((-86.92184 36.11929, -86.92099 36.118..."
9,10,Bellevue - Hwy 100,Third,,Center,"POLYGON ((-86.95118 36.04780, -86.95088 36.047..."


In [39]:
nash_next['FeatureType'].unique()

array(['Anchor Park', 'Center', 'Transition or Infill', 'Special Uses',
       'Neighborhood', 'Green network'], dtype=object)

In [40]:
nash_next[nash_next['FeatureType'] == 'Green network'].shape

(151, 6)